### Selected Ngram Repeat With Youtube Link

In [2]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from nltk import ngrams
from functools import reduce
from pathlib import Path
import shutil

In [3]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [4]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 1000  # 28 native word end index

In [5]:
def df_col_value_join_comma(df, df_columns_list):
    '''
    df_col_value_join_comma(df_test, ["video_id","start_time","end_time"])\n
    function used for selected column value join with comma in one row
    '''
    column_value_list = []
    for column in df_columns_list:
        list_var = df[f"{column}"].to_list()
        list_var_string = [str(x) for x in list_var] 
        list_var_join = ",".join(list_var_string)
        column_value_list.append(list_var_join)

    df_result = pd.DataFrame([[column_value_list[0],column_value_list[1],column_value_list[2]]], columns=df_columns_list)

    return df_result

In [6]:
#data_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
#Deployment3/Data/Manuel Select"

In [7]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment3/Result/4-Selected Ngram Repeat With Youtube Link"

Path(path).mkdir(parents=True, exist_ok=True)

##### Link Count

In [24]:
# file extention
file_ext = "Threegram"

In [25]:
path_folder = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment3/Result/Ngram Youtube Link Manuel"
file = f"{lang_folder.capitalize()}_{file_ext}_With_{word_end}_Word_Youtube_Link_Manuel_12.xlsx"   ########*****
#sheet = "Sheet1"  # Sheet1

In [26]:
#df_link_selected_manuel = pd.read_excel(f"{path_folder}/{file}", sheet_name=f"{sheet}")
df_link_selected_manuel = pd.read_excel(f"{path_folder}/{file}")
df_link_selected_manuel

,video_id,start_time,end_time,text
0,7L64XN3or3I,30,32,hiçbir fikrim yok
1,yfbaI_fKgQc,230,232,hiçbir fikrim yok
2,OtTPQ4877Hk,462,464,hiçbir fikrim yok
3,xagFSGk8vT4,371,373,hiçbir fikrim yok
4,L3bf7dlR5hk,384,386,hiçbir fikrim yok
...,...,...,...,...
766,vA4ywYo4QbY,1962,1964,böyle bir şey
767,5EsyiXMKOPs,197,199,böyle bir şey
768,6rqwPW97HiY,495,497,böyle bir şey
769,h93wXNLXmYU,895,897,böyle bir şey


In [27]:
# disabled because of youtube tools
#df_link_selected_manuel["video_url"] = "https://www.youtube.com/watch?v="+df_link_selected_manuel['video_id'].map(str)+"&t="+df_link_selected_manuel['start_time'].map(str)+"s"
#df_link_selected_manuel

In [28]:
df_count = df_link_selected_manuel.groupby(["text"])[["text"]].count()
df_count.rename(columns={"text":"count"}, inplace=True)
df_count.sort_values(by="count", ascending=False, inplace=True)
df_count.reset_index(inplace=True)
df_count

,text,count
0,bir daha asla,9
1,evet ben de,9
2,ne olursa olsun,9
3,her şey yolunda,9
4,çok büyük bir,9
...,...,...
164,fark eder ki,1
165,ne yaptığını sanıyorsun,1
166,nerede olduğunu biliyor,1
167,beni duyuyor musun,1


In [29]:
df_count = df_count[df_count["count"] > 2]

In [30]:
len(df_count)

143

In [31]:
df_count_merge = pd.merge(df_link_selected_manuel, df_count, how="inner",on= "text")
df_count_merge.sort_values(by=["count","text"], ascending=False, inplace=True)
df_count_merge.reset_index(drop=True, inplace=True)
df_count_merge

,video_id,start_time,end_time,text,count
0,nPVd4XaeM34,1032,1034,çok büyük bir,9
1,fCAke_9xf1g,455,457,çok büyük bir,9
2,_J2Lhghabe0,99,101,çok büyük bir,9
3,RIWR28A9mMc,71,73,çok büyük bir,9
4,9LbSz7aJn7Y,355,357,çok büyük bir,9
...,...,...,...,...,...
726,0pYtgRxceX8,331,333,benim için çok,3
727,haycX_ZM4a4,740,742,benim için çok,3
728,yiSWKv_9JkE,12,14,benim için bir,3
729,GFVhV5BlTOA,91,93,benim için bir,3


In [47]:
df_count.to_excel(f"{path_folder}/{file_ext} Video List.xlsx", index=False)

In [48]:
df_count_merge.to_excel(f"{path_folder}/{lang_folder.capitalize()}_{file_ext}_With_{word_end}_Word_Youtube_Link_Manuel_Count.xlsx", index=False)   ########*****

##### Link Repeat

In [24]:
# ******  
path_folder_file = f"{path_folder}/{lang_folder.capitalize()}_{file_ext}_With_{word_end}_Word_Youtube_Link_Manuel_Count_Select.xlsx"   # from youtube tool

In [25]:
#df_word_group_time_loc = pd.read_excel(f"{path_folder_file2}", sheet_name=f"{sheet2}")
df_word_group_time_loc = pd.read_excel(f"{path_folder_file}")
#df_word_group_time_loc = pd.read_excel(f"Twogram Threegram Youtube Link Selected_Manuel.xlsx")   
df_word_group_time_loc

,video_id,start_time,end_time,text
0,T5krOQidV5s,714,716,ne yapabilirim
1,c90800j_ED4,1066,1068,ne yapabilirim
2,Ts4gUJI0Iks,986,988,ne yapabilirim
3,Ssg7vpw8Zb4,427,429,ne yapabilirim
4,M0SXeJ45hww,371,373,ne yapabilirim
...,...,...,...,...
907,m3Clq7uulPg,4093,4095,bir bak
908,GQgS2ruEJAs,5182,5184,bir bak
909,n2t0ueWtUNI,54,56,başka bir
910,N0a3f_R2BfQ,790,792,başka bir


In [26]:
df_count2 = df_word_group_time_loc.groupby(["text"])[["text"]].count()
df_count2.rename(columns={"text":"count"}, inplace=True)
df_count2.sort_values(by="count", ascending=False, inplace=True)
df_count2.reset_index(inplace=True)
df_count2

,text,count
0,ne demek,7
1,kadar çok,7
2,tam olarak,7
3,var biliyor,7
4,iyi olacak,7
...,...,...
170,yok mu,3
171,evet ben,3
172,kontrol altında,3
173,söyledi mi,3


In [27]:
df_word_group_time_loc = pd.merge(df_word_group_time_loc, df_count2, how="left", on="text")
df_word_group_time_loc.drop_duplicates(inplace=True)
df_word_group_time_loc

,video_id,start_time,end_time,text,count
0,T5krOQidV5s,714,716,ne yapabilirim,7
1,c90800j_ED4,1066,1068,ne yapabilirim,7
2,Ts4gUJI0Iks,986,988,ne yapabilirim,7
3,Ssg7vpw8Zb4,427,429,ne yapabilirim,7
4,M0SXeJ45hww,371,373,ne yapabilirim,7
...,...,...,...,...,...
907,m3Clq7uulPg,4093,4095,bir bak,3
908,GQgS2ruEJAs,5182,5184,bir bak,3
909,n2t0ueWtUNI,54,56,başka bir,3
910,N0a3f_R2BfQ,790,792,başka bir,3


In [28]:
search_list = df_word_group_time_loc.loc[:,"text"].unique()
len(search_list)

175

In [29]:
# other option 
# mUf7VNqChac =>  black screen
# 0_CDMstFg7M => 10sn
# bj1JRuyYeco => 20sn
# cElhIDdGz7M => screensaver
default_video_id = "Q-8I-uMUMYA"
#df_link_default = pd.DataFrame(data=[["repeat",5,7,"repeat_again",f"{default_video_id}",f"https://www.youtube.com/watch?v={default_video_id}&t=0s"]], columns=["text","start_time","end_time","sentence","video_id","video_url"])
df_link_default = pd.DataFrame(data=[["repeat",5,7,"repeat_again",f"{default_video_id}"]], columns=["text","start_time","end_time","sentence","video_id"])
df_link_default

,text,start_time,end_time,sentence,video_id
0,repeat,5,7,repeat_again,Q-8I-uMUMYA


In [30]:
#(df_result_repeat["end_time"].sum()-df_result_repeat["start_time"].sum())/60

In [31]:
# Each word group result convert to join result in one row
df_all_join_result = pd.DataFrame()
seq_num = 0  # option
for word_group in search_list:
    df_result_repeat = pd.DataFrame()
    # for repeat 1
    df_word_group_search_repeat1 = df_word_group_time_loc[df_word_group_time_loc["text"] == word_group]
    # for repeat 2
    try:
        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["text"] == word_group].sample(1)
    except:
        df_word_group_search_repeat2 = df_word_group_time_loc[df_word_group_time_loc["text"] == word_group].tail(1)
    df_word_group_search_repeat2.reset_index(drop=True, inplace=True)   
    # repeat block
    for i in range(len(df_word_group_search_repeat1)):        
        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat1.iloc[[i,]]], axis=0)

    try:
        df_link_default_var = df_link_default
        word_time_diff_var = df_word_group_search_repeat2.loc[0 ,"end_time"] - df_word_group_search_repeat2.loc[0 ,"start_time"]
        if word_time_diff_var < 4:
            word_time_diff_var2 = 3
        else:
            word_time_diff_var2 = word_time_diff_var+1.0

        df_link_default_var.loc[0,"end_time"] = df_link_default_var.loc[0,"start_time"] + word_time_diff_var2
    except:
        pass

    for j in range(3):
        df_result_repeat = pd.concat([df_result_repeat,df_word_group_search_repeat2], axis=0)
        df_result_repeat = pd.concat([df_result_repeat,df_link_default_var], axis=0)
        df_result_repeat.reset_index(drop=True, inplace=True)

    df_result_repeat_var = df_col_value_join_comma(df_result_repeat, ["video_id","start_time","end_time"]) 
    df_result_repeat_var.loc[0,["text"]] = f"{word_group}"  # option for search string
    df_result_repeat_var.loc[0,["count"]] = df_result_repeat.loc[0,"count"] # option for count num     
    df_all_join_result = pd.concat([df_all_join_result,df_result_repeat_var], axis=0)

df_all_join_result.reset_index(drop=True, inplace=True)
df_all_join_result["web_format"] = f"[play_lms_videos video_id="+df_all_join_result["video_id"].map(str)+" start_\
time="+df_all_join_result["start_time"].map(str)+" end_time="+df_all_join_result["end_time"].map(str)+"]"
df_all_join_result.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Youtube_Talk_Time_Join_{file_ext}.xlsx", index=False)

In [32]:
df_all_join_result

,video_id,start_time,end_time,text,count,web_format
0,"T5krOQidV5s,c90800j_ED4,Ts4gUJI0Iks,Ssg7vpw8Zb...","714,1066,986,427,371,4220,187,187,5,187,5,187,5","716,1068,988,429,373,4222,189,189,8,189,8,189,8",ne yapabilirim,7.0,"[play_lms_videos video_id=T5krOQidV5s,c90800j_..."
1,"pvsImT6E4yM,d4O2uEPSgjo,Ts4gUJI0Iks,Z34xp6mjK4...","5224,1747,426,87,126,208,967,1747,5,1747,5,1747,5","5226,1749,428,89,128,210,969,1749,8,1749,8,1749,8",öyle bir,7.0,"[play_lms_videos video_id=pvsImT6E4yM,d4O2uEPS..."
2,"_bt4XuwqSvU,cUbw3HKKSr8,0fBopIv7q3o,Prrt7mTtbe...","76,18,83,207,221,186,557,221,5,221,5,221,5","78,20,85,209,223,188,559,223,8,223,8,223,8",ne olduğunu,7.0,"[play_lms_videos video_id=_bt4XuwqSvU,cUbw3HKK..."
3,"7fc3_6Uycl0,s4MqASVecXc,1X1tr7ctRUg,zJCqecTO14...","278,926,67,307,4679,2389,4679,5,4679,5,4679,5","280,928,69,309,4681,2391,4681,8,4681,8,4681,8",tamam mı,6.0,"[play_lms_videos video_id=7fc3_6Uycl0,s4MqASVe..."
4,"K5GIun0flM4,PXbpdKbcgD0,LsbRXTusV6E,KN15Qezt3q...","2091,12,1737,216,162,50,2136,1737,5,1737,5,1737,5","2093,14,1739,218,164,52,2138,1739,8,1739,8,1739,8",onun gibi,7.0,"[play_lms_videos video_id=K5GIun0flM4,PXbpdKbc..."
...,...,...,...,...,...,...
170,"eX9znWdbp8s,q6blniyuZ5o,7PGLODg3Xg4,eX9znWdbp8...","170,585,153,170,5,170,5,170,5","172,587,155,172,8,172,8,172,8",bir kez,3.0,"[play_lms_videos video_id=eX9znWdbp8s,q6blniyu..."
171,"lREyKzqpCjE,Uo5B3AGNE_c,DJXee_4POCA,DJXee_4POC...","4418,226,539,539,5,539,5,539,5","4420,228,541,541,8,541,8,541,8",bir fikir,3.0,"[play_lms_videos video_id=lREyKzqpCjE,Uo5B3AGN..."
172,"TXbhyTyHmyA,lsciLyjICMk,IbNzJFmtkZ0,lsciLyjICM...","813,3953,5729,3953,5,3953,5,3953,5","815,3955,5730,3955,8,3955,8,3955,8",bir dakika,3.0,"[play_lms_videos video_id=TXbhyTyHmyA,lsciLyjI..."
173,"7gw4cgfRxPM,m3Clq7uulPg,GQgS2ruEJAs,7gw4cgfRxP...","2868,4093,5182,2868,5,2868,5,2868,5","2870,4095,5184,2870,8,2870,8,2870,8",bir bak,3.0,"[play_lms_videos video_id=7gw4cgfRxPM,m3Clq7uu..."


#### Copy Move And Delete

In [49]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{word_end}_Word_Youtube_Talk_Time_Join_*.xlsx")
output_file

['Turkish_1000_Word_Youtube_Talk_Time_Join_Twogram.xlsx']

In [132]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [133]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass